In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math

data = pd.read_csv("dataAfterPOS.csv")
data

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,aromas tropical fruit broom brimstone herb pal...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,ripe fruity wine firm tannins juicy red berry ...,Avidagos,87,15.000000,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,tart snappy flavors lime flesh dominate green ...,unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,pineapple rind lemon pith orange blossom aroma...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,much regular bottling rough tannic rustic eart...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41
5,Spain,blackberry typical navarran whiff green herbs ...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,unknown,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,42
6,Italy,bright informal aromas candied berry white pep...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,31
7,France,dry restrained wine offers profusion acidity f...,unknown,87,24.000000,Alsace,Alsace,unknown,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,21
8,Germany,savory thyme notes accent sunnier flavors pres...,Shine,87,12.000000,Rheinhessen,unknown,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,24
9,France,great depth flavor fresh apple pear fruits tou...,Les Natures,87,27.000000,Alsace,Alsace,unknown,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,30


# pre processing

In [2]:
#remove punctuation
data['description'] = data['description'].str.replace('[^\w\s]','')

In [3]:
#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq

wine       72465
flavors    56228
fruit      40686
palate     33691
aromas     32164
acidity    31672
finish     29689
tannins    26477
cherry     23568
ripe       23310
dtype: int64

In [4]:
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [6]:
unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

30993


In [7]:
unfreqList = unfreq[-count:]
#unfreqList

In [8]:
unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

# TF IDF  fulltext da ignorare

fullText = pd.Series(' '.join(data['description']).split()).value_counts()
#fullText

tfData = {}
for idx, row in data.iterrows(): 
    tfData[idx] = pd.Series(row['description'].split()).value_counts();

tf1 = {}
for idx in tfData.keys():
    tf1[idx] = {}
    for word in tfData[idx].keys():
        tf1[idx][word] = tfData[idx][word] / fullText[word]


#idf
docCount = dict.fromkeys(fullText.keys(),0)
idfDict1 = {}
N = len(data)                                #numero di documenti #dizionario contenente tutte le parole presenti 
idfData = dict(tfData)
for el in idfData:     #per ogni descrizione
    for word,val in idfData[el].items():
        docCount[word] = docCount[word] + 1
        
for word in fullText.keys():
    idfDict1[word] = math.log10(N/docCount[word])  

#tf-idf
tfIdf1 = {}
for idx in tfData.keys():
    tfIdf1[idx] = {}
    for word in tfData[idx].keys():
        val = tf1[idx][word] * idfDict1[word]
        tfIdf1[idx][word] = val
tfIdf1[0]

!!!! tf idf devono essere usati sulle varietà, così si capisce per quali qualità sono i vini più importanti. NOn si calcola il "vettore" delle parole nella descrizione mas i cercano le parole nella descrizione e si vedono che peso hanno in quella generale.

calcolari in generale non ha senso, la frequenza delle parole all'interno di uno short text cime questo è sempre 1. 

# tf idf contando l'importanza di certi termini all'interno di una classe di vini
(raw dataset)

In [9]:
fullText = pd.Series(' '.join(data['description']).split()).value_counts()


In [10]:
totalWords = pd.Series("".join(data["description"]).split()).value_counts().sum()
print(totalWords)
t1 = data.groupby(["variety"])
t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()
t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
t2 = t2.sort_values('word_count',ascending = False)
#t2.iloc[0]["text"]
#t2

1777428


In [11]:
varietyDict = {}
for idx, row in t2.iterrows():
    varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()
#varietyDict["Pinot Gris"]

In [12]:
tfDict = {}
for var in varietyDict:
    temp = {}
    for word in varietyDict[var].keys():
        temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
    tfDict[var] = temp
#tfDict["Pinot Gris"] 

In [13]:
varList = data["variety"].unique().tolist()

def computeIdf1(dataset):
    a = []
    idf = {}
    for idx,row in dataset.iterrows():
        el = row["variety"]
        docList = data[data["variety"] == el]
        N = len(docList)
        idfDict = {}
        temp = t2[t2["variety"] == el]
        idfDict = dict.fromkeys(temp.iloc[0]["text"].split(),0)
        
        for idx,row in docList.iterrows():
            for word in row["description"].split():
                idfDict[word] += 1
        
        for word, val in idfDict.items():
            idfDict[word] = math.log10(N/float(val))
        a.append([el,N,idfDict])    
        idf[el] = idfDict
    return idf, a

In [14]:
idf,a = computeIdf1(t2)

In [15]:
a[-40] 

['Torontel',
 1,
 {'elegant': 0.0,
  'apricot': 0.0,
  'peach': 0.0,
  'mango': 0.0,
  'spice': 0.0,
  'bouquet': 0.0,
  'fruity': 0.0,
  'fullbodied': 0.0,
  'sticky': 0.0,
  'proper': 0.0,
  'cinnamon': 0.0,
  'caramel': 0.0,
  'toffee': 0.0,
  'pear': 0.0}]

C'è un problema coi vini che hanno solo una descrizione, li si leva? idf = 0.

In [16]:
tfIdf = {}
for var in varietyDict:
    temp = {}
    for word in tfDict[var].keys():
        temp[word] = tfDict[var][word] * idf[var][word]
    tfIdf[var] = temp 

In [17]:
dictlist = []
for word,value in tfIdf["Chardonnay"].items():
    temp = [word,value]
    dictlist.append(temp)
dictlist.sort(key=lambda x: x[1],reverse=True)
dictlist
#puligny,chablisienne è un produttore porcodio, bisogna togliere i nomi dei produttori dalle recensioni e il nome della varietà di vino su cui si sta lavorando?

[['chablisienne', 3.0300732307125178],
 ['chablisian', 3.0300732307125178],
 ['puligny', 3.0300732307125178],
 ['fourchaume', 2.9161298784056813],
 ['chablislike', 2.825953248056593],
 ['chablisstyle', 2.799624309334244],
 ['browned', 2.7682476491458736],
 ['serein', 2.751319629759689],
 ['pulignymontrachet', 2.7396238334247682],
 ['fuissé', 2.7290432350485365],
 ['cortoncharlemagne', 2.7239002935368246],
 ['kumeu', 2.7239002935368246],
 ['clarified', 2.5288835385459403],
 ['cob', 2.525061025593765],
 ['chassagnemontrachet', 2.3308255620865523],
 ['meursault', 2.300896646534136],
 ['scallop', 2.2741696317967817],
 ['chards', 2.230732681258936],
 ['whistles', 2.2299816717219914],
 ['panseared', 2.153107236614547],
 ['bells', 2.1404864787426274],
 ['challah', 2.084655495813716],
 ['movie', 2.084655495813716],
 ['guavas', 2.082982577410513],
 ['brie', 2.0552387258593408],
 ['leflaive', 2.0385585511834545],
 ['mâcon', 2.0215135074433603],
 ['butteredtoast', 2.0200488204750116],
 ['daffodil

# tf idf con stemming 

In [18]:
data = pd.read_csv("stemmed.csv")
#remove punctuation
data['description'] = data['description'].str.replace('[^\w\s]','')


In [19]:
#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq
#numeri sono più alti

wine      74984
flavor    62401
fruit     52898
acid      35519
palat     33793
aroma     33436
finish    31497
tannin    28259
cherri    27524
ripe      24822
dtype: int64

In [20]:
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [21]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [22]:
unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

26449


In [23]:
unfreqList = unfreq[-count:]
#unfreqList

In [24]:
unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

In [25]:
fullText = pd.Series(' '.join(data['description']).split()).value_counts()
fullText

black           22598
note            22353
rich            19885
spice           19749
fresh           19087
dri             17090
red             16397
berri           15821
oak             15116
nose            14395
plum            13697
sweet           13315
blackberri      13182
appl            12914
soft            12706
textur          12602
drink           11787
crisp           11740
blend           11667
white           10922
fruiti          10804
light           10388
dark            10214
citru            9952
hint             9823
cabernet         9736
herb             9607
vanilla          9536
bright           9402
raspberri        9228
                ...  
maresh             10
undergo            10
smidgeon           10
herblac            10
labor              10
peachflavor        10
lemonrind          10
gregorio           10
satisfactori       10
saintbri           10
contradict         10
k                  10
penedè             10
medit              10
strasser  

In [26]:
totalWords = pd.Series("".join(data["description"]).split()).value_counts().sum()
print(totalWords)
t1 = data.groupby(["variety"])
t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()
t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
t2 = t2.sort_values('word_count',ascending = False)
#t2.iloc[0]["text"]
#t2

1752815


In [27]:
varietyDict = {}
for idx, row in t2.iterrows():
    varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()
#varietyDict["Pinot Gris"]

In [28]:
tfDict = {}
for var in varietyDict:
    temp = {}
    for word in varietyDict[var].keys():
        temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
    tfDict[var] = temp
#tfDict["Pinot Gris"] 

In [29]:
varList = data["variety"].unique().tolist()

def computeIdf1(dataset):
    a = []
    idf = {}
    for idx,row in dataset.iterrows():
        el = row["variety"]
        docList = data[data["variety"] == el]
        N = len(docList)
        idfDict = {}
        temp = t2[t2["variety"] == el]
        idfDict = dict.fromkeys(temp.iloc[0]["text"].split(),0)
        
        for idx,row in docList.iterrows():
            for word in row["description"].split():
                idfDict[word] += 1
        
        for word, val in idfDict.items():
            idfDict[word] = math.log10(N/float(val))
        a.append([el,N,idfDict])    
        idf[el] = idfDict
    return idf, a

In [30]:
idf,a = computeIdf1(t2)

In [31]:
a[-40] 

['Uvalino',
 1,
 {'rare': 0.0,
  'nativ': 0.0,
  'grape': 0.0,
  'black': 0.0,
  'plum': 0.0,
  'vanilla': 0.0,
  'dens': 0.0,
  'robust': 0.0,
  'deliv': 0.0,
  'clove': 0.0,
  'dust': 0.0,
  'cocoa': 0.0,
  'assert': 0.0,
  'enjoy': 0.0}]

C'è un problema coi vini che hanno solo una descrizione, li si leva? idf = 0.

In [32]:
tfIdf = {}
for var in varietyDict:
    temp = {}
    for word in tfDict[var].keys():
        temp[word] = tfDict[var][word] * idf[var][word]
    tfIdf[var] = temp 

In [33]:
dictlist = []
for word,value in tfIdf["Chardonnay"].items():
    temp = [word,value]
    dictlist.append(temp)
dictlist.sort(key=lambda x: x[1],reverse=True)
dictlist
#con lo stemming cambia un po' le cose

[['chablisian', 3.0300732307125178],
 ['chablisienn', 3.0300732307125178],
 ['puligni', 3.0300732307125178],
 ['fourchaum', 2.9161298784056813],
 ['chablislik', 2.825953248056593],
 ['chablisstyl', 2.799624309334244],
 ['serein', 2.751319629759689],
 ['pulignymontrachet', 2.7396238334247682],
 ['kumeu', 2.7239002935368246],
 ['cortoncharlemagn', 2.7239002935368246],
 ['fuissé', 2.7078539359785987],
 ['clarifi', 2.5288835385459403],
 ['cob', 2.525061025593765],
 ['chassagnemontrachet', 2.3308255620865523],
 ['meursault', 2.2930796076703244],
 ['pansear', 2.153107236614547],
 ['movi', 2.129421268573749],
 ['challah', 2.084655495813716],
 ['brie', 2.0552387258593408],
 ['leflaiv', 2.0385585511834545],
 ['mâcon', 2.0215135074433603],
 ['butteredtoast', 2.0200488204750116],
 ['durel', 1.9672613231099287],
 ['butterscotchi', 1.9338521177116013],
 ['brewer', 1.9242973807511228],
 ['brulé', 1.9242973807511228],
 ['went', 1.9195136010542828],
 ['marcona', 1.9092969679895715],
 ['daffodil', 1.89